In [1]:
import pandas as pd

In [13]:
df = pd.read_csv('/workspaces/vikrant_dubai/final/AJ/AJ 1.csv')
unique_station_names = df['Station Name'].unique()


In [14]:
def create_finder_table(df):
    df['vlookup'] = df['Station Name'].str[:3] + df['Date'].str.replace('/', '')
    df_grouped = df.groupby(['Station Name', 'Date']).first().reset_index()
    df_grouped['vlookup'] = df_grouped['Station Name'].str[:5] + df_grouped['Date'].str.replace('/', '')
    Finder_table = df_grouped[['Station Name', 'Date', 'vlookup']]
    return Finder_table

# Example usage:
Finder_table = create_finder_table(df)
Finder_table.to_csv('/workspaces/vikrant_dubai/SQL/Finder_table.csv', index=False)

In [4]:
pollutants = ['SO2', 'NO2','CO','PM10', 'PM2.5']

# Convert pollutant columns to numeric, forcing errors to NaN
df[pollutants] = df[pollutants].apply(pd.to_numeric, errors='coerce')

avg_values = df.groupby(['Station Name', 'Date'])[pollutants].mean().reset_index()

final_df = pd.merge(avg_values, Finder_table, on=['Station Name', 'Date'], suffixes=(' Avg 24H', ''))
final_df.rename(columns={'Station Name': 'Name'}, inplace=True)
final_df['vlookup'] = final_df['Name'].str[:5] + final_df['Date'].str.replace('/', '')
final_df=final_df.drop(columns=['Name', 'Date'])
final_df

,SO2,NO2,CO,PM10,PM2.5,vlookup
0,1.908333,28.313750,1.162500,1.059583,0.085417,Ajman01012023
1,1.879583,31.040000,0.802917,15.120833,4.544167,Ajman02012023
2,1.812917,33.769583,0.669167,19.547500,6.686250,Ajman03012023
3,1.757500,21.143333,0.549583,26.630833,7.845833,Ajman04012023
4,1.766250,22.423333,0.714167,51.584583,17.287083,Ajman05012023
...,...,...,...,...,...,...
150,3.962750,9.141500,0.144582,7.840588,5.583529,Masfo27012023
151,5.349167,16.708500,0.183709,NaN,NaN,Masfo28012023
152,6.779250,25.693333,0.207567,NaN,NaN,Masfo29012023
153,6.517250,24.534000,0.241923,NaN,NaN,Masfo30012023


In [5]:
selected_columns = df[['Station Name', 'Date', 'O3']]


# Extract unique station names
unique_station_names = selected_columns["Station Name"].unique()

# Define the process_data function
def process_data(values, dates):
    # Create DataFrame
    df_temp = pd.DataFrame({'Index': range(len(values)), 'Values': values, 'Date': dates})

    # Calculate 8-hour moving average
    df_temp['8h_Avg'] = df_temp['Values'].rolling(window=8).mean()

    # Calculate datacap (percentage of available data in 8-hour window)
    df_temp['datacap'] = df_temp['Values'].rolling(window=8).count() / 8 * 100

    # Ensure first 7 values of 'datacap' are blank (NaN)
    df_temp.loc[:6, 'datacap'] = None

    # Define function to calculate 8h_Avg_WHO and 8h_Avg_NS
    def calculate_adjusted_avg(df_temp, max_threshold):
        adjusted_avg = [None] * len(df_temp)
        for i in range(7, len(df_temp)):
            if df_temp.loc[i, 'datacap'] > 75:
                adjusted_avg[i] = df_temp.loc[i, '8h_Avg']
            else:
                max_value = df_temp.loc[i-7:i, 'Values'].max()
                if max_value > max_threshold:
                    adjusted_avg[i] = max_value
        return adjusted_avg

    # Calculate 8h_Avg_WHO with max value threshold of 100
    df_temp['8h_Avg_WHO'] = calculate_adjusted_avg(df_temp, 100)

    # Calculate 8h_Avg_NS with max value threshold of 120
    df_temp['8h_Avg_NS'] = calculate_adjusted_avg(df_temp, 120)

    return df_temp

# Process data for each station name
dfs = []
for station in unique_station_names:
    station_data = selected_columns[selected_columns['Station Name'] == station]
    station_values = station_data["O3"].tolist()
    station_dates = station_data["Date"].tolist()
    processed_df = process_data(station_values, station_dates)
    processed_df['Station Name'] = station
    dfs.append(processed_df)

# Concatenate all DataFrames
df2 = pd.concat(dfs, ignore_index=True)

# Rearrange the columns as specified
df2 = df2[['Station Name', 'Date', 'Values', 'datacap', '8h_Avg_WHO', '8h_Avg_NS']]
df2.rename(columns={'Values': 'O3'}, inplace=True)

# Convert 'Date' column to datetime format for proper grouping
df2['Date'] = pd.to_datetime(df2['Date'], dayfirst=True)

# Calculate 8h_Avg_WHO_DC and 8h_Avg_NS_DC
df2['8h_Avg_WHO_DC'] = df2.groupby(['Station Name', 'Date'])['8h_Avg_WHO'].transform(lambda x: x.count() / 24 * 100)
df2['8h_Avg_NS_DC'] = df2.groupby(['Station Name', 'Date'])['8h_Avg_NS'].transform(lambda x: x.count() / 24 * 100)

# Ensure only the last row of each day for each station retains the values, setting others to NaN
df2.loc[df2.duplicated(subset=['Station Name', 'Date'], keep='last'), ['8h_Avg_WHO_DC', '8h_Avg_NS_DC']] = None

# Group by 'Station Name' and 'Date'
grouped = df2.groupby(['Station Name', 'Date'])

# Create new columns for the calculated daily averages
df2['Daily_Avg_8h_WHO'] = None
df2['Daily_Avg_8h_NS'] = None

# Iterate over each group to calculate the averages where count > 75
for (station, date), group in grouped:
    if group['8h_Avg_WHO_DC'].sum() > 75:  # Check if total count > 75 for WHO
        avg_value_who = group['8h_Avg_WHO'].mean()  # Calculate average for WHO
        df2.loc[group.index[-1], 'Daily_Avg_8h_WHO'] = avg_value_who  # Assign to last row
    
    if group['8h_Avg_NS_DC'].sum() > 75:  # Check if total count > 75 for NS
        avg_value_ns = group['8h_Avg_NS'].mean()  # Calculate average for NS
        df2.loc[group.index[-1], 'Daily_Avg_8h_NS'] = avg_value_ns  # Assign to last row



# Convert 'Date' column to datetime format
df2['Date'] = pd.to_datetime(df2['Date'], dayfirst=True)

# Change date format to ddmmyyyy
df2['Date'] = df2['Date'].dt.strftime('%d/%m/%Y')

# Add a new column 'vlookup' that takes the first 5 letters of 'Station Name' and concatenates it with 'Date'
df2['vlookup'] = df2['Station Name'].str[:5] + df2['Date'].str.replace('/', '')

# Remove the 'Station Name' column
df2.drop(columns=['Station Name','Date'], inplace=True)


last_rows_per_date_station = df2.drop_duplicates(subset=['vlookup'], keep='last')

last_rows_per_date_station = last_rows_per_date_station[['vlookup'] + [col for col in last_rows_per_date_station.columns if col != 'vlookup']]

# Reset the index
last_rows_per_date_station.reset_index(drop=True, inplace=True)

last_rows_per_date_station

,vlookup,O3,datacap,8h_Avg_WHO,8h_Avg_NS,8h_Avg_WHO_DC,8h_Avg_NS_DC,Daily_Avg_8h_WHO,Daily_Avg_8h_NS
0,Masfo01012023,109.9392,100.0,127.3626,127.3626,70.833333,70.833333,None,None
1,Masfo02012023,96.1968,100.0,117.7920,117.7920,100.000000,100.000000,102.648775,102.648775
2,Masfo03012023,104.0496,100.0,117.7920,117.7920,100.000000,100.000000,112.832875,112.832875
3,Masfo04012023,109.9392,100.0,118.0374,118.0374,100.000000,100.000000,113.896275,113.896275
4,Masfo05012023,86.3808,100.0,108.9576,108.9576,100.000000,100.000000,107.659025,107.659025
...,...,...,...,...,...,...,...,...,...
150,Jurf27012023,47.1168,100.0,51.7794,51.7794,100.000000,100.000000,66.60565,66.60565
151,Jurf28012023,3.9264,100.0,32.8836,32.8836,100.000000,100.000000,52.812125,52.812125
152,Jurf29012023,3.9264,100.0,28.7118,28.7118,100.000000,100.000000,37.5871,37.5871
153,Jurf30012023,1.9632,100.0,58.1598,58.1598,100.000000,100.000000,57.822375,57.822375


In [6]:
selected_columns = df[['Station Name', 'Date', 'CO']]

df=selected_columns

# Convert 'Date' column to datetime format for proper processing
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)

# Extract unique station names
unique_station_names = df["Station Name"].unique()

# Define the process_data function for CO transformation
def process_data(values, dates):
    # Create DataFrame
    df_temp = pd.DataFrame({'Index': range(len(values)), 'CO': values, 'Date': dates})

    # Calculate 8-hour moving average
    df_temp['C0_8h_Avg'] = df_temp['CO'].rolling(window=8).mean()

    # Calculate datacap (percentage of available data in 8-hour window)
    df_temp['datacap'] = df_temp['CO'].rolling(window=8).count() / 8 * 100

    # Ensure first 7 values of 'datacap' are blank (NaN)
    df_temp.loc[:6, 'datacap'] = None

    return df_temp

# Process data for each station name
dfs = []
for station in unique_station_names:
    station_data = df[df['Station Name'] == station]
    station_values = station_data["CO"].tolist()
    station_dates = station_data["Date"].tolist()
    processed_df = process_data(station_values, station_dates)
    processed_df['Station Name'] = station
    dfs.append(processed_df)

# Concatenate all DataFrames
df2 = pd.concat(dfs, ignore_index=True)

# Rearrange the columns as specified
df2 = df2[['Station Name', 'Date', 'CO', 'datacap', 'C0_8h_Avg']]

# Calculate C0_8h_Avg_DC (daily data capture percentage for CO 8h Avg)
df2['C0_8h_Avg_DC'] = df2.groupby(['Station Name', 'Date'])['C0_8h_Avg'].transform(lambda x: x.count() / 24 * 100)

# Ensure only the last row of each day for each station retains the values, setting others to NaN
df2.loc[df2.duplicated(subset=['Station Name', 'Date'], keep='last'), ['C0_8h_Avg_DC']] = None

# Group by 'Station Name' and 'Date'
grouped = df2.groupby(['Station Name', 'Date'])

# Create new columns for the calculated daily averages
df2['Daily_Avg_8h_C0'] = None

# Iterate over each group to calculate the daily averages where count > 75
for (station, date), group in grouped:
    if group['C0_8h_Avg_DC'].sum() > 75:  # Check if total count > 75
        avg_value_c0 = group['C0_8h_Avg'].mean()  # Calculate average
        df2.loc[group.index[-1], 'Daily_Avg_8h_C0'] = avg_value_c0  # Assign to last row


# Convert 'Date' column to datetime format
df2['Date'] = pd.to_datetime(df2['Date'], dayfirst=True)

# Change date format to ddmmyyyy
df2['Date'] = df2['Date'].dt.strftime('%d/%m/%Y')

# Add a new column 'vlookup' that takes the first 5 letters of 'Station Name' and concatenates it with 'Date'
df2['vlookup'] = df2['Station Name'].str[:5] + df2['Date'].str.replace('/', '')

# Remove the 'Station Name' column
df2.drop(columns=['Station Name','Date'], inplace=True)


last_rows_per_date_station2 = df2.drop_duplicates(subset=['vlookup'], keep='last')

last_rows_per_date_station2 = last_rows_per_date_station2[['vlookup'] + [col for col in last_rows_per_date_station2.columns if col != 'vlookup']]

# Reset the index
last_rows_per_date_station2.reset_index(drop=True, inplace=True)

last_rows_per_date_station2

,vlookup,CO,datacap,C0_8h_Avg,C0_8h_Avg_DC,Daily_Avg_8h_C0
0,Masfo01012023,0.137424,100.0,0.186095,70.833333,None
1,Masfo02012023,0.160328,100.0,0.154602,100.000000,0.175836
2,Masfo03012023,0.148876,100.0,0.135992,100.000000,0.158419
3,Masfo04012023,0.125972,100.0,0.141718,100.000000,0.14142
4,Masfo05012023,0.137424,100.0,0.134561,100.000000,0.138617
...,...,...,...,...,...,...
150,Jurf27012023,0.286300,100.0,0.247649,100.000000,0.193014
151,Jurf28012023,0.824544,100.0,0.518203,100.000000,0.281648
152,Jurf29012023,2.336208,100.0,1.371377,100.000000,0.649305
153,Jurf30012023,0.984872,100.0,0.770147,100.000000,0.835519


In [7]:
# Merge last_rows_per_date_station and last_rows_per_date_station2 on 'vlookup'
combined_df = pd.merge(last_rows_per_date_station, last_rows_per_date_station2, on='vlookup', suffixes=('_O3', '_CO'))

# Merge the result with final_df on 'vlookup'
combined_df = pd.merge(combined_df, final_df, on='vlookup')

# Display the combined dataframe
D24hparameter=combined_df[['vlookup','SO2', 'NO2', 'Daily_Avg_8h_WHO', 'Daily_Avg_8h_NS', 'Daily_Avg_8h_C0', 'PM10', 'PM2.5']]

D24hparameter.to_csv('/workspaces/vikrant_dubai/SQL/D24hparameter.csv', index=False)

In [8]:
# Round all columns of D24hparameter to 1 decimal place
D24hparameter_rounded = D24hparameter.round(1)
# Convert 'Daily_Avg_8h_WHO', 'Daily_Avg_8h_NS', and 'Daily_Avg_8h_C0' to numeric, forcing errors to NaN
D24hparameter_rounded[['Daily_Avg_8h_WHO', 'Daily_Avg_8h_NS', 'Daily_Avg_8h_C0']] = D24hparameter_rounded[['Daily_Avg_8h_WHO', 'Daily_Avg_8h_NS', 'Daily_Avg_8h_C0']].apply(pd.to_numeric, errors='coerce')

# Round the specified columns to 1 decimal place
D24hparameter_rounded[['Daily_Avg_8h_WHO', 'Daily_Avg_8h_NS', 'Daily_Avg_8h_C0']] = D24hparameter_rounded[['Daily_Avg_8h_WHO', 'Daily_Avg_8h_NS', 'Daily_Avg_8h_C0']].round(1)
# Display the new table
D24hparameter_rounded.to_csv('/workspaces/vikrant_dubai/SQL/D24hparameter_rounded.csv', index=False)    

In [9]:
df=D24hparameter_rounded

# Create a new DataFrame KPI_NS containing only the 'vlookup' column
KPI_NS = df[['vlookup']].copy()

# Add 'SO2' column based on the condition
KPI_NS['SO2'] = df['SO2'].apply(lambda x: 1 if x < 150.4 else 0)

# Add 'NO2' column based on the condition
KPI_NS['NO2'] = df['NO2'].apply(lambda x: 1 if x < 150.4 else 0)

# Add 'O3' column based on the condition using 'Daily_Avg_8h_NS'
KPI_NS['O3'] = df['Daily_Avg_8h_NS'].apply(lambda x: 1 if x < 120.4 else 0)

# Add 'CO' column based on the condition using 'Daily_Avg_8h_C0'
KPI_NS['CO'] = df['Daily_Avg_8h_C0'].apply(lambda x: 1 if x < 10.4 else 0)


KPI_NS.to_csv('/workspaces/vikrant_dubai/SQL/KPI_NS.csv', index=False)

In [10]:
df=D24hparameter_rounded

# Create a new DataFrame KPI_NS containing only the 'vlookup' column
KPI_WHO = df[['vlookup']].copy()

# Add 'SO2' column based on the condition
KPI_WHO['SO2'] = df['SO2'].apply(lambda x: 1 if x < 20.4 else 0)

# Add 'NO2' column based on the condition
KPI_WHO['NO2'] = df['NO2'].apply(lambda x: 1 if x < 75.4 else 0)

# Add 'O3' column based on the condition using 'Daily_Avg_8h_NS'
KPI_WHO['O3'] = df['Daily_Avg_8h_NS'].apply(lambda x: 1 if x < 100.4 else 0)

# Add 'CO' column based on the condition using 'Daily_Avg_8h_C0'
KPI_WHO['CO'] = df['Daily_Avg_8h_C0'].apply(lambda x: 1 if x < 10.4 else 0)


KPI_WHO.to_csv('/workspaces/vikrant_dubai/SQL/KPI_WHO.csv', index=False)

In [11]:
KPI_PM = df[['vlookup']].copy()
KPI_PM['PM10'] = df['PM10'].apply(lambda x: 1 if x < 150.4 else 0)
KPI_PM['PM2.5'] = df['PM2.5'].apply(lambda x: 1 if x < 60.4 else 0)
KPI_PM.to_csv('/workspaces/vikrant_dubai/SQL/KPI_PM.csv', index=False)